In [473]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt
import torchtext

import tiktoken
import pandas as pd
import numpy as np

from torch.nn import functional as F
import torch.nn as nn
from tqdm.notebook import tqdm
import pickle
import ast

In [406]:
import os

#dataset https://nlp.stanford.edu/projects/nmt/

#tiktoken api https://github.com/openai/tiktoken
cl100k_base = tiktoken.get_encoding("cl100k_base")

# In production, load the arguments directly instead of accessing private attributes
# See openai_public.py for examples of arguments for specific encodings
tokenizer = tiktoken.Encoding(
    # If you're changing the set of special tokens, make sure to use a different name
    # It should be clear from the name what behaviour to expect.
    name="cl100k_im",
    pat_str=cl100k_base._pat_str,
    mergeable_ranks=cl100k_base._mergeable_ranks,
    special_tokens={
        **cl100k_base._special_tokens,
        "<|PAD|>": 0,
        "<|START|>": 100278,
        "<|END|>": 100279,
        "<|DEL|>": 100280,
        "!": 100281
    }
)
print(tokenizer.n_vocab) #this is the number of tokens in our tokenizer
print(tokenizer._special_tokens) #prints out our special tokens 

specials = {"<|PAD|>","<|START|>","<|END|>", "<|DEL|>", "!"}

100282
{'<|endoftext|>': 100257, '<|fim_prefix|>': 100258, '<|fim_middle|>': 100259, '<|fim_suffix|>': 100260, '<|endofprompt|>': 100276, '<|PAD|>': 0, '<|START|>': 100278, '<|END|>': 100279, '<|DEL|>': 100280, '!': 100281}


In [589]:
#GLOBALS

block_size = 64 #This is the value of T
batch_size = 16 #This it the value of B
n_embed = 512
dropout = 0.2
n_heads = 8
n_layers = 6

vocab_size = tokenizer.n_vocab
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device is: {device}")

device is: cuda


In [590]:
def create_dataset():
    num_examples = 100000

    en_max = 0
    en_length = []
    with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
        sentences_en = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            sentence = "<|PAD|> " + (line) + " <|PAD|>"
            tok_sentence = tokenizer.encode(sentence, allowed_special = specials)
            en_length.append(len(tok_sentence))
            if len(tok_sentence) > en_max:
                en_max = len(tok_sentence)
                print(en_max)

            if len(tok_sentence) <= block_size:
                len_pad = block_size - len(tok_sentence)
                tok_sentence = tok_sentence + len_pad*[0]
                assert len(tok_sentence) == block_size, print(len(tok_sentence))
                sentences_en.append(tok_sentence)
            else:
                sentences_en.append(block_size*[100280])

    en_length = torch.tensor(en_length).float()     
    print(en_max)    
    print(f"Length of sentences: {len(sentences_en)}")

    de_max = 0
    de_length = []
    with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
        sentences_de = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            sentence = "<|START|> " + (line) + " <|END|>"
            tok_sentence = tokenizer.encode(sentence, allowed_special = specials)
            de_length.append(len(tok_sentence))
            if len(tok_sentence) > de_max:
                de_max = len(tok_sentence)
                print(de_max)

            if len(tok_sentence) <= block_size:
                len_pad = block_size - len(tok_sentence)
                tok_sentence = tok_sentence + len_pad*[0]
                assert len(tok_sentence) == block_size, print(len(tok_sentence))
                sentences_de.append(tok_sentence)
            else:
                sentences_de.append(block_size*[100280])

    de_length = torch.tensor(de_length).float()               
    print(de_max) 
    print(f"Length of sentences: {len(sentences_de)}")

    print("Removing sentences whos length is greater than our block_size")

    #combine the arrays together
    sentences = np.array([sentences_en, sentences_de])
    #check for indices in both sentences that have rows containing the DEL token
    idx = np.where(sentences == 100280)

    #delete every row that contains the DEL token
    sentences = np.delete(sentences, idx[1], axis = 1)

    #splitting to german and english

    sentences_en = torch.tensor(sentences[0], dtype=torch.long)
    sentences_de = torch.tensor(sentences[1], dtype=torch.long)

    print(f"Length of new english sentences: {len(sentences_en)}")
    print(f"Length of new german sentences: {len(sentences_de)}")

    print(f"Average length of english tokenized sentence: {torch.mean(en_length)}, and with std: {torch.std(en_length)}")
    print(f"Average length of german tokenized sentence: {torch.mean(de_length)}, and with std: {torch.std(de_length)}")

    with open(os.getcwd()+'\\data\\english_sentences.pkl', 'wb') as f:
        pickle.dump(sentences_en, f)

    with open(os.getcwd()+'\\data\\german_sentences.pkl', 'wb') as f:
        pickle.dump(sentences_de, f)


In [591]:
create = True
if create:
    create_dataset()

  0%|          | 0/100000 [00:00<?, ?it/s]

38
50
60
67
68
82
95
122
125
132
141
154
177
191
203
295
295
Length of sentences: 100000


  0%|          | 0/100000 [00:00<?, ?it/s]

60
73
83
91
105
120
130
135
161
163
187
239
267
284
284
Length of sentences: 100000
Removing sentences whos length is greater than our block_size
Length of new english sentences: 83474
Length of new german sentences: 83474
Average length of english tokenized sentence: 32.97871017456055, and with std: 17.489957809448242
Average length of german tokenized sentence: 42.1722297668457, and with std: 22.630157470703125


In [565]:
#TRAIN AND VAL DATASETS

with open(os.getcwd()+'\\data\\english_sentences.pkl', 'rb') as f:
    english_sentences = pickle.load(f)

with open(os.getcwd()+'\\data\\german_sentences.pkl', 'rb') as f:
    german_sentences = pickle.load(f)

In [566]:
100280 in english_sentences

False

In [569]:
english_sentences[50]

tensor([    0,  1472, 18217,  9507,   387, 14792,  1268,  4228,  3489,   847,
          706,  1903,   433,   311, 15405,   279,  2539,  4754,   315,   682,
          701,  2955,  3241,   662,   220,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])

In [613]:
#BATCH LOADER
n = int(0.9*len(english_sentences))

train_data_en = english_sentences[:n]
val_data_en = english_sentences[n:]

train_data_de = german_sentences[:n]
val_data_de = german_sentences[n:]

def get_batch(split):
    xdata = train_data_en if "train" else train_data_en
    ydata = train_data_de if "train" else val_data_de
    idx = torch.randint(len(xdata), (batch_size,))
    print(idx)
    x = torch.stack([xdata[i] for i in idx])
    y = torch.stack([ydata[i] for i in idx])

    #shifting our targets by 1 to the right
    y = y[:, 1:]
    #to pad the last dimension of the input tensor, pad has the form (padding_left, padding_right)
    y = F.pad(input = y, pad = (0,1,0,0), mode = 'constant', value = 0)

    # x, y = x.to(device), y.to(device)

    return x, y

#xb, yb = get_batch('train')

In [499]:
#Pytorch's positional encoding https://pytorch.org/tutorials/beginner/transformer_tutorial.html
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
            x: (T, B, C)
            We have to change our shape dimensions in to (T, B, C) and then change it back to (B, T, C) when done

        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [634]:
class Head(nn.Module):
    def __init__(self, head_size, decoder = False):
        super().__init__()
        self.decoder = decoder
        self.head_size = head_size
        self.Wk = nn.Linear(n_embed, head_size)
        self.Wq = nn.Linear(n_embed, head_size)
        self.Wv = nn.Linear(n_embed, head_size)

        if self.decoder:
            self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #assume input is of size (B, T, C)
        K = self.Wk(x) #(B, T, head_size)
        Q = self.Wq(x) #(B, T, head_size)
        V = self.Wv(x) #(B, T, head_size)

        attention_scores = Q @ K.transpose(-2, -1) * 1/(self.head_size)**(1/2) #(B, T, T)

        if self.decoder:
            attention_scores = attention_scores.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B, T, T)

        attention_scores = F.softmax(attention_scores, dim = -1) #(B, T, T)
        scores = self.dropout(attention_scores) #(B, T, T)
        out = scores @ V #(B, T, T) @ (B, T, head_size) = (B, T, head_size)

        return out

class crossHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.Wk = nn.Linear(n_embed, head_size)
        self.Wq = nn.Linear(n_embed, head_size)
        self.Wv = nn.Linear(n_embed, head_size)

        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, enc_out):
        #assume x is of shape (B, T, C)
        #assume enc_out is of shape (B, T, C)

        K = self.Wk(enc_out) #(B, T, head_size)
        Q = self.Wq(x) #(B, T, head_size)
        V = self.Wv(enc_out) #(B, T, head_size)

        attention_scores = Q @ K.transpose(-2, -1) * 1/(self.head_size)**(1/2) #(B, T, T)
        attention_scores = attention_scores.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B, T, T)
        attention_scores = F.softmax(attention_scores, dim = -1) #(B, T, T)
        scores = self.dropout(attention_scores) #(B, T, T)
        out = scores @ V #(B, T, head_size)

        return out


class MultiHeadSelfAttention(nn.Module):
    def __init__(self, head_size, decoder):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, decoder) for _ in range(n_heads)])
        #output of heads is of size (B, T, n_heads*head_size)
        self.proj = nn.Linear(head_size * n_heads, n_embed)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.proj(out)
        out = self.dropout(out)

        return out

class MultiHeadCrossAttention(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.heads = nn.ModuleList([crossHead(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_heads*head_size, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out):
        x = torch.cat([h(x, enc_out) for h in self.heads], dim = -1)
        x = self.proj(x)
        x = self.dropout(x)

        return x

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(n_embed, 4*n_embed)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4*n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)

        return x

class EncoderBlock(nn.Module):
    def __init__(self):
        super().__init__()
        head_size = n_embed // n_heads

        self.sa = MultiHeadSelfAttention(head_size, decoder = False)
        self.ffw = FeedForward()
        self.layernorm1 = nn.LayerNorm(n_embed)
        self.layernorm2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        #assume input x is of size (B, T, C)
        x = self.layernorm1(x) #(B, T, C)
        x = x + self.sa(x) #(B, T, C)
        x = self.layernorm2(x) #(B, T, C)
        x = x = self.ffw(x) #(B, T, C)

        return x

class DecoderCrossBlock(nn.Module):
    #one implementation of the multi head cross attention block in the decoder
    def __init__(self):
        super().__init__()
        head_size = n_embed // n_heads
        self.heads = MultiHeadCrossAttention(head_size)
        self.layernorm1 = nn.LayerNorm(n_embed)
        self.layernorm2 = nn.LayerNorm(n_embed)
        self.ffw = FeedForward()

    def forward(self, x, enc_out):
        #assume x is input of shape (B, T, C), x is the output of the decoder self attention layer
        x = self.layernorm1(x) #(B, T, C)
        x = x + self.heads(x, enc_out) #(B, T, C)
        x = self.layernorm2(x) #(B, T, C)
        x = x + self.ffw(x) #(B, T, C)

        return x

class DecoderSelfBlock(nn.Module):
    #one implementation of the multi head self attention block in the decoder
    def __init__(self):
        super().__init__()
        head_size = n_embed//n_heads
        self.sa = MultiHeadSelfAttention(head_size, decoder = True)
        self.layernorm1 = nn.LayerNorm(n_embed)
        self.layernorm2 = nn.LayerNorm(n_embed)
        self.ffw = FeedForward()

    def forward(self, x):
        #assume x is of shape (B, T, C)
        x = self.layernorm1(x)
        x = x + self.sa(x)
        x = self.layernorm2(x)
        x = x + self.ffw(x)

        return x

class Transformer(nn.Module):

    def __init__(self, n_layers):
        super().__init__()
        self.tok_embedding_matrix_x = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding_x = PositionalEncoding(n_embd, dropout = dropout)

        self.tok_embedding_matrix_y = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding_y = PositionalEncoding(n_embd, dropout = dropout)

        self.EncoderBlocks = nn.Sequential(*[EncoderBlock() for _ in range(n_layers)])
        self.DecoderSelfBlocks = nn.Sequential(*[DecoderSelfBlock() for _ in range(n_layers)])
        self.DecoderCrossBlocks = nn.Sequential(*[DecoderCrossBlock() for _ in range(n_layers)])

        self.final_layer_norm = nn.LayerNorm(n_embd)
        self.final_linear = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, input, target):
        tok_embed_x = self.tok_embedding_matrix_x(input)
        pos_embed_x = self.pos_embedding_x(token_embed_x.view(T,B,C)).view(B, T, C)

        tok_embed_y = self.tok_embedding_matrix_y(input)
        pos_embed_y = self.pos_embedding_y(token_embed_y.view(T,B,C)).view(B, T, C)

        x = tok_embed_x + pos_embed_x
        y = tok_embed_y + pos_embed_y

        #encoder
        enc_out = self.EncoderBlocks(x)

        #decoder self
        y = self.DecoderSelfBlocks(y)
        
        #decoder cross
        y = self.CrossAttentionBlocks(y, enc_out)


rough work

In [614]:
xb, yb = get_batch('train')

tensor([17123, 63621, 52424, 36214, 10675, 67899, 46588, 73345,  7587, 18486,
        63459, 17373,  1692, 56649,   482, 37312])


In [615]:
yb.device

device(type='cpu')

In [616]:
tok_embedding_matrix = nn.Embedding(vocab_size, n_embed)
pos_embedding = PositionalEncoding(n_embed, dropout = dropout)

In [617]:
token_embed_x = tok_embedding_matrix(xb) #(B, T, C)
B, T, C = token_embed_x.shape
pos_embed_x = pos_embedding(token_embed_x.view(T,B,C)).view(B, T, C) #(B, T, C)

input = token_embed_x + pos_embed_x #(B, T, C)

token_embed_y = tok_embedding_matrix(yb) #(B, T, C)
B, T, C = token_embed_y.shape
pos_embed_y = pos_embedding(token_embed_y.view(T,B,C)).view(B, T, C) #(B, T, C)

target = token_embed_y + pos_embed_y #(B, T, C)

In [618]:
input.shape

torch.Size([16, 64, 512])

In [619]:
e = Head(head_size = 64, decoder = False)
e = MultiHeadSelfAttention(64, decoder = False)
e = EncoderBlock()

In [638]:
e(input).shape

out_e = e(input)

In [635]:
d = Head(head_size = 64, decoder = True)
d = MultiHeadSelfAttention(head_size = 64, decoder = True)
d = DecoderSelfBlock()

In [639]:
d(target).shape

out_d = d(target)

In [642]:
c = crossHead(head_size = 64)
c = MultiHeadCrossAttention(head_size = 64)

In [643]:
c(out_d, out_e)

tensor([[[-1.5961e-01, -0.0000e+00,  2.3434e-02,  ...,  9.1408e-02,
           1.2809e-02, -1.3358e-02],
         [-5.2937e-02, -0.0000e+00,  6.2386e-02,  ..., -0.0000e+00,
           6.4907e-03,  6.2555e-02],
         [ 2.4392e-02, -3.1080e-02,  8.6604e-02,  ..., -1.2029e-01,
           6.8772e-03,  8.7691e-02],
         ...,
         [-5.8023e-03, -1.3832e-01,  4.6914e-02,  ...,  6.0742e-02,
          -5.9813e-02, -2.8790e-02],
         [-6.2929e-03, -1.3868e-01,  4.7524e-02,  ...,  0.0000e+00,
          -0.0000e+00, -0.0000e+00],
         [-9.3312e-03, -1.5363e-01,  0.0000e+00,  ...,  7.3021e-02,
          -0.0000e+00, -3.3025e-02]],

        [[-5.6573e-02, -3.2562e-01,  1.0243e-01,  ...,  2.4677e-01,
          -0.0000e+00, -8.8965e-02],
         [-6.4727e-02,  5.4738e-02,  1.4060e-01,  ...,  6.0478e-02,
          -7.0306e-02, -0.0000e+00],
         [-2.2432e-03,  2.9734e-02,  1.2461e-01,  ..., -4.5891e-03,
          -0.0000e+00, -1.2388e-01],
         ...,
         [ 2.2815e-02, -1

In [301]:
encoder = tiktoken.get_encoding("cl100k_base")


In [403]:
tokenizer.encode("<|PAD|>", allowed_special = specials)

[0]

In [303]:
encoding.decode([83, 1609, 5963, 374, 2294, 0])

'tiktoken is great!'

In [416]:
block_size = 10
num_examples = 5

en_max = 0 
with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
    idx_en = []
    sentences_en = []
    for i in tqdm(range(num_examples)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        sentence = "<|PAD|> " + (line) + " <|PAD|>"
        print(sentence)
        print(tokenizer.encode(sentence, allowed_special = specials))
        tok_sentence = tokenizer.encode(sentence, allowed_special = specials)
        print(len(tok_sentence))
        if len(tok_sentence) > en_max:
            en_max = len(tok_sentence)
            print(en_max)

        if len(tok_sentence) <= block_size:
            len_pad = block_size - len(tok_sentence)
            tok_sentence = tok_sentence + len_pad*[100277]
            assert len(tok_sentence) == block_size, print(len(tok_sentence))
            #idx_en.append(i)
            sentences_en.append(tok_sentence)
        else:
            sentences_en.append(block_size*[100280])

print(en_max)    
print(f"Length of sentences: {len(sentences_en)}")


de_max = 0 
with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
    idx_de = []
    sentences_de = []
    for i in tqdm(range(num_examples)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        sentence = "<|START|> " + (line) + " <|END|>"
        tok_sentence = tokenizer.encode(sentence, allowed_special = specials)
        if len(tok_sentence) > de_max:
            de_max = len(tok_sentence)
            print(de_max)

        if len(tok_sentence) <= block_size:
            len_pad = block_size - len(tok_sentence)
            tok_sentence = tok_sentence + len_pad*[100277]
            assert len(tok_sentence) == block_size, print(len(tok_sentence))
            #idx_en.append(i)
            sentences_de.append(tok_sentence)
        else:
            sentences_de.append(block_size*[100280])
            
print(de_max)  
print(f"Length of sentences: {len(sentences_de)}")

  0%|          | 0/5 [00:00<?, ?it/s]

<|PAD|> iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould . <|PAD|>
[0, 11245, 24532, 374, 264, 5644, 369, 1005, 25982, 902, 374, 17551, 439, 264, 1488, 1169, 555, 2231, 1919, 22145, 477, 14654, 304, 279, 51370, 13116, 320, 24359, 883, 315, 279, 9699, 6892, 354, 51370, 662, 220, 0]
38
38
<|PAD|> iron cement protects the ingot against the hot , abrasive steel casting process . <|PAD|>
[0, 11245, 24532, 36236, 279, 6892, 354, 2403, 279, 4106, 1174, 94804, 9699, 25146, 1920, 662, 220, 0]
18
<|PAD|> a fire restant repair cement for fire places , ovens , open fireplaces etc . <|PAD|>
[0, 264, 4027, 2800, 519, 13023, 24532, 369, 4027, 7634, 1174, 297, 21778, 1174, 1825, 4027, 27170, 5099, 662, 220, 0]
21
<|PAD|> Construction and repair of highways and ... <|PAD|>
[0, 24987, 323, 13023, 315, 60395, 323, 2564, 220, 0]
10
<|PAD|> An announcement must be commercial character . <|PAD|>
[0, 1556, 17480, 2

  0%|          | 0/5 [00:00<?, ?it/s]

60
60
Length of sentences: 5


In [417]:
np.array(sentences_en).shape

(5, 10)

In [418]:
sentences = np.array([sentences_en, sentences_de])

In [419]:
sentences = np.array([sentences_en, sentences_de])

idx = np.where(sentences == 100280)
print(idx[1])

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4
 4 4 4 4 4 4]


In [420]:
sentences = np.delete(sentences, idx[1], axis = 1)

In [421]:
sentences

array([], shape=(2, 0, 10), dtype=int32)